In [3]:
import pandas as pd
import numpy as np
import scipy.stats as ss

In [4]:
def compare_economic_growth(state_GDP, USA_GDP, state_control):
    
    '''
    Parameters
    ----------
    state_GDP : pd.DataFrame()
        Annual GDP growth for each state in the US
    USA_GDP : pd.DataFrame()
        United States annual GDP growth
    state_control : pd.DataFrame()
        Trifecta status for each US state and year of last status change
    
    Outputs
    -------
    republican_states : pd.DataFrame()
        Republican states with a statistically significant difference in GDP growth, relative to US GDP growth, after becoming a trifecta government
        Including p value and difference in state GPD growth
    '''

    republican_states = pd.DataFrame()
    democratic_states = pd.DataFrame()

    for index, row in state_control.iterrows():

        try:

            state_data = state_GDP[['observation_date', row['Abbreviation']]]
            state_data.columns.values[1] = 'GDP'

            state_data['observation_date'] = state_data.apply(lambda row: int(row['observation_date']), axis=1)

            state_data = state_data.merge(USA_GDP, on='observation_date',how='left')

            state_data['relative_to_USA_GDP'] = state_data.apply(lambda row: row['GDP'] - row['USA_GDP'], axis=1)

            switch_year = row['Year of last status change']

            state_data_before = state_data[state_data['observation_date'] <= switch_year]['relative_to_USA_GDP']
            state_data_after = state_data[state_data['observation_date'] > switch_year]['relative_to_USA_GDP']
            
            state_data_before = state_data_before.dropna()
            state_data_after = state_data_after.dropna()

            tstat, p_val = ss.ttest_ind(state_data_before, state_data_after, equal_var=False)

            if p_val <= 0.05:

                new_df = pd.DataFrame(data = {'state':row['Abbreviation'],'p_val':[p_val], 'difference':[state_data_after.mean() - state_data_before.mean()],'years_of_data': 2022-switch_year})

                if 'Republican trifecta' in row['Trifecta status']:
                    republican_states = pd.concat([republican_states, new_df])
                elif 'Democratic trifecta' in row['Trifecta status']:
                    democratic_states = pd.concat([democratic_states, new_df])

            else:
                continue

        except Exception as e:
            pass

    return republican_states, democratic_states